# Langchain
## Introduction
LangChain is an open-source framework for developing applications powered by large language models (LLMs). A core purpose is to act as the "glue" that connects various components, including LLMs, to build reliable applications. It provides a standard, consistent interface for interacting with different LLM providers.
LangChain's core components are 
- chains for linking multiple LLMs or tools
- agents for enabling LLMs to interact with external resources
- messages, defined by its role and its content, for structuring conversation input and output. 

## Connecting to LLMs via a chat model
LangChain can work with OpenAI, Cohere, Bloom, Huggingface, DeepSeek, and others. This makes LangChain model-agnostic, allowing developers to choose the best model for their needs while benefiting from LangChain's architecture.

- Official models: These are models that are officially supported by LangChain and/or model provider. You can find these models in the langchain-<provider> packages.
- Community models: There are models that are mostly contributed and supported by the community. You can find these models in the langchain-community package.

For our test we will use the langchain_openai, wich need in the environment a variable called OPENAI_API_KEY with your api key

In [1]:
import dotenv
from langchain_openai import ChatOpenAI
dotenv.load_dotenv()
chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

### Direct questioning
Old school, valid only for testing purposes. Its done directly calling to the chat_model  using the `.invoke()` method to pass a query. This will return an AIMessage object (we will talk about message roles just below)

In [2]:
response = chat_model.invoke("Whats blood pressure?")
print(response.content)

Blood pressure is the force exerted by circulating blood against the walls of the body's arteries, the major blood vessels in the body. It is one of the vital signs used to assess the overall health of an individual. Blood pressure is typically expressed in terms of two measurements: systolic and diastolic pressure.

1. **Systolic Pressure**: This is the higher number and measures the pressure in the arteries when the heart beats and pumps blood.

2. **Diastolic Pressure**: This is the lower number and measures the pressure in the arteries when the heart is at rest between beats.

Blood pressure is usually recorded as two numbers, written as a ratio, such as 120/80 mmHg (millimeters of mercury). The first number is the systolic pressure, and the second is the diastolic pressure.

Normal blood pressure for most adults is typically around 120/80 mmHg. However, what is considered normal can vary based on age, health conditions, and other factors. High blood pressure, or hypertension, is a

### Messages and roles
LLMs are typically accessed through a chat model interface that takes a list of messages as input and returns a message as output. 

Messages are the unit of communication, representing model input and output, and are typically associated with roles like "system," "human," or "assistant". LangChain provides its own unified message format (e.g., SystemMessage, HumanMessage, AIMessage) 

- System Message: Used to prime the behavior of the AI model and provide additional context. It instructs the model on how to behave or sets the tone. For example, it can tell the model to act as an expert on a specific topic or to only answer questions within a certain domain. Not all providers support a dedicated system message role, but LangChain attempts to adapt by including the content in a human message or using a separate API parameter if supported. In LangChain's unified format, this is represented by the SystemMessage class.

- Human Message: Represents input from a user interacting with the model. This is typically in the form of text, but some chat models also support multimodal content like images or audio. LangChain automatically converts a simple string input into a HumanMessage object for convenience. This is represented by the HumanMessage class in LangChain.

- Assistant Message: Represents a response from the model. This response can include text or a request for the model to invoke tools. Multimodal outputs are possible but still uncommon. In LangChain's format, this is represented by the AIMessage class. When streaming responses, partial messages are returned as AIMessageChunk objects, which can be aggregated into a single AIMessage.

- Tool Message: Used to pass the results of executing a tool back to the model. After a model requests to use a tool (via the tool_calls attribute in an AIMessage), this message type contains the output from that tool's execution. It includes a tool_call_id field linking it back to the specific tool call request. This is represented by the ToolMessage class.

- Function Message (Legacy): This is a legacy message type that corresponded to OpenAI's previous function-calling API. The ToolMessage should be used instead for the updated tool-calling API. Represented by the Legacy FunctionMessage class.

Besides these roles and their content, messages can also contain other data like a unique identifier (id), an optional name to differentiate speakers, metadata (like token usage), and importantly, tool_calls. The tool_calls attribute within an AIMessage represents a request from the model to call one or more tools, distinct from the ToolMessage which contains the result of that call.

In [3]:
from langchain.schema.messages import HumanMessage, SystemMessage

messages = []
environment = SystemMessage(
        content="""You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."""
    )
question  = HumanMessage(content="What is blood pressure?")
messages.append(environment)
messages.append(question)
chat_model.invoke(messages)

AIMessage(content="Blood pressure is the force exerted by circulating blood against the walls of the body's arteries, the major blood vessels in the body. It is one of the principal vital signs used to assess the health of an individual. Blood pressure is typically expressed in terms of two measurements: systolic and diastolic pressure.\n\n1. **Systolic Pressure**: This is the higher number and represents the pressure in the arteries when the heart beats and fills them with blood.\n\n2. **Diastolic Pressure**: This is the lower number and represents the pressure in the arteries when the heart is at rest between beats.\n\nBlood pressure is measured in millimeters of mercury (mmHg) and is recorded with the systolic number first, followed by the diastolic number, for example, 120/80 mmHg.\n\nNormal blood pressure is generally considered to be around 120/80 mmHg. High blood pressure, or hypertension, is a condition where the blood pressure is consistently too high, which can lead to health

In [4]:
question  = HumanMessage(content="How to change a tire?")
messages.append(environment)
messages.append(question)
chat_model.invoke(messages)


AIMessage(content="I'm here to help with healthcare-related questions. If you have any questions about health or medical topics, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 56, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_76544d79cb', 'id': 'chatcmpl-BbuAIupkW2zR6WsW6ZVzAZMTraV2N', 'finish_reason': 'stop', 'logprobs': None}, id='run-9ceb8e1b-6925-4183-86a1-e2a6740aa006-0', usage_metadata={'input_tokens': 56, 'output_tokens': 25, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Prompt Templates
As seen, modern Chat Models require a list of messages with specific roles, a simple PromptTemplate (which formats a single string) is insufficient for constructing the full input sequence in a structured way. This is where ChatPromptTemplate comes in. It is a type of prompt template specifically designed to format a list of messages.
### simple prompt template

In [5]:
from langchain.prompts import PromptTemplate

template_str = """You're an expert on {topic}. ...
Here is an user review:
{context}

Answer the following question in less than 10 words
{question}"""
prompt_template = PromptTemplate.from_template(template_str)
prompt_template.input_variables

['context', 'question', 'topic']

In [6]:
# same filler, different template
filled_prompt = prompt_template.format(
    topic="User feedback", context="I love it here!", question="Is this a positive review?"
)
chat_model.invoke(filled_prompt)

AIMessage(content='Yes, it is a positive review.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 43, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_76544d79cb', 'id': 'chatcmpl-BbuATUzeDsHP8qUX37VcYaYHZJr6b', 'finish_reason': 'stop', 'logprobs': None}, id='run-32375033-249e-4fbd-be90-014a304d3efb-0', usage_metadata={'input_tokens': 43, 'output_tokens': 8, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### system msg, human msg and chat prompt templates
We saw that there are different roles for the messages, tipically, the system and human messages. There are specific prompt templates that generate role colored messages:
- SystemMessagePromptTemplate
- HumanMessagePromptTemplate

All the messages can be joined in a ChatPromptTemplate to replace all its variables and send it through the invoke method.

This ready-to-be-filled PromptTemplate templates can be created with the `from_template()` function wich will locate and create the parameters of the template automatically
```python
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate.from_template(review_system_template_str)
)
``` 
or invoking directly the creator:
```python
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=review_template_str,
    )
)
```


In [15]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

Patient reviews:

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate.from_template(review_system_template_str)
)
print (review_system_prompt)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate.from_template("{question}")
)
print(review_human_prompt)

prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nPatient reviews:\n\n{context}\n") additional_kwargs={}
prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}') additional_kwargs={}


In [8]:

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate.from_messages(messages)
review_prompt_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nPatient reviews:\n\n{context}\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [9]:
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

filled_prompt = review_prompt_template.format_messages(
    context=context, question=question
)

print(chat_model.invoke(filled_prompt))

content='Yes, one patient mentioned having a great stay, which indicates a positive experience.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 86, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9bddfca6e2', 'id': 'chatcmpl-BbuAlMQZz6uyDheTOC67BSXzwncV7', 'finish_reason': 'stop', 'logprobs': None} id='run-5f44eb0a-6bc7-447a-b186-61b20d7748b4-0' usage_metadata={'input_tokens': 86, 'output_tokens': 16, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## Chains
Chains are a core concept in LangChain, representing an end-to-end wrapper around multiple individual components executed in a defined order. They allow developers to go beyond a single API call to an LLM and instead chain together multiple calls in a logical sequence. This enables breaking down complex tasks into smaller steps, maintaining context between calls by feeding the output of one step as input to the next, and adding intermediate processing logic. 

LangChain offers various types of chains:

- LLM Chain: The simplest form, consisting of a PromptTemplate, a language model (LLM or ChatModel), and an optional output parser. We can see here the steps and how they feed on the output of the previous one: 
    - takes the input parameters
    - uses the PromptTemplate to format them into a prompt
    - passes the prompt to the model
    - optionally uses the OutputParser to refine the result.
- Sequential Chain: Combines various individual chains where the output of one chain serves as the input for the next in a continuous sequence. There are two types: Simple Sequential Chains, which handle a single input and output, and a more general form that allows for multiple inputs/outputs.
- Router Chain: Used for complicated tasks when there are multiple subchains specialized for different types of input. It consists of a Router Chain that selects the next chain, Destination Chains that it can route to, and a Default Chain used when the router cannot decide. This adds intelligent decision-making by directing inputs to the most suitable processing paths.
- Transform Chain: Applies a data transformation between chains.

In [10]:
# define the chain
review_chain = review_prompt_template | chat_model

context = "I had a great stay!"
question = "Did the patient have a positive experience?"
# invoke it passing the parameters of the template for the first step
print(review_chain.invoke({"context": context, "question": question}))

question = "what was he interned for?"
print(review_chain.invoke({"context": context, "question": question}))

context = "I need my car fixed. it has a punctured tire"
question = "how to fix his car"
print(review_chain.invoke({"context": context, "question": question}))

content='Yes, the patient had a positive experience, as indicated by their statement, "I had a great stay!"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 87, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_76544d79cb', 'id': 'chatcmpl-BbuApcPrCxYIG9IIFpYuoGr73gvqj', 'finish_reason': 'stop', 'logprobs': None} id='run-79694fe5-992f-4880-b778-86c140faded7-0' usage_metadata={'input_tokens': 87, 'output_tokens': 22, 'total_tokens': 109, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content="I'm sorry, the context provided does not include information about the reason for the patient's stay or what they were interned for." addit

### Inspecting a chain
For complex chains it will be good to have on your toolbelt the langchain_core.globals > set_debug

In [11]:
from langchain_core.globals import set_debug
set_debug(True)

print(review_chain.invoke({"context": context, "question": question}))
set_debug(False)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "context": "I need my car fixed. it has a punctured tire",
  "question": "how to fix his car"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "context": "I need my car fixed. it has a punctured tire",
  "question": "how to fix his car"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nPatient reviews:\n\nI need my car fixed. it has a punctured tire\n\nHuman: how to fix his car"
  ]
}
[llm/end] [chain:Ru

### Output parser
So far we have in our chain a ChatPromptTemplate (review_prompt_template) and an LLM, in our case a ChatOpenAI (chat_model): We can add to the end a third block, an output parser to ease the reading 

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

# redefine the chain
review_chain = review_prompt_template | chat_model | output_parser

#fill the parameters and invoke
context = "I had a great stay! I had terrible stomach pain. I ache for 3 days and a stone was found and removed from my liver"
question = "How was his stay?"
response = review_chain.invoke({"context": context, "question": question})
print(response)


The patient described their stay as great.


In [13]:
from reviews import reviews
response = review_chain.invoke({"context": reviews, "question": "tell me the total number of reviews, the number of pure positive ones, the number of puere negative ones and the number of mixed ones that you have access to ?"})
print(response)

The total number of reviews I have access to is 24.

Pure Positive Reviews: These reviews are overwhelmingly positive without mentioning any significant drawbacks.
1. Review by Christy Johnson
2. Review by Anna Frazier
3. Review by Abigail Mitchell
4. Review by Cody Ibarra
5. Review by Scott Terry

Pure Negative Reviews: These reviews are overwhelmingly negative without mentioning any positive aspects.
1. Review by Rachel Carter

Mixed Reviews: These reviews contain both positive and negative aspects.
1. Review by Kimberly Rivas
2. Review by Catherine Yang
3. Review by Jennifer Russell
4. Review by Henry Hays
5. Review by Kim Franklin (two reviews)
6. Review by Michael Smith (two reviews)
7. Review by Chelsea Mitchell
8. Review by Carol Byrd
9. Review by Daniel Williams
10. Review by Kim Powers
11. Review by Sharon Brown
12. Review by John Bartlett
13. Review by Rebecca Wilkerson
14. Review by Michele Jones
15. Review by Tiffany Long
16. Review by Stacy Villa

In summary:
- Total Revie

In [14]:
response = review_chain.invoke({"context": reviews, "question": "who is the doctor most mentioned and what are the comments about him, good or bad?"})
print(response)

The doctor most mentioned in the reviews is Kyle Vasquez, with two reviews. The comments about him are as follows:

1. Review by Kim Franklin (Visit ID: 3332): "The hospital provided exceptional care, and the nursing staff was incredibly supportive. However, the administrative processes were a bit convoluted, causing some confusion." This review is generally positive about the care provided but notes an issue with the administrative processes.

2. Another review by Kim Franklin (Visit ID: 3332): "The hospital staff was efficient, and the facilities were clean. However, the lack of communication about my treatment plan was frustrating." This review is mixed, highlighting the efficiency and cleanliness but expressing frustration over communication issues regarding the treatment plan.
